# Chapter 21. Network Analysis

In [19]:
from __future__ import division
import math, random, re
from collections import defaultdict, Counter, deque
from linear_algebra import dot, get_row, get_column, make_matrix, magnitude, scalar_multiply, shape, distance
from functools import partial

Many interesting data problems can be thought of in terms of [networks](https://en.wikipedia.org/wiki/Network_theory) consisting of nodes of some type and the edges that join them (see [graphs](https://en.wikipedia.org/wiki/Graph_%28discrete_mathematics%29) and [graph theory](https://en.wikipedia.org/wiki/Graph_theory) for more information on nodes and edges).  
For example, your Facebook friends form the nodes of a network whose edges are friendship relations.  
A less obvious example is the World Wide Web itself, with each web page a node, and each hyperlink from one page to another an edge.  
Facebook friendship is mutual -- if I am Facebook friends with you then necessarily you are friends with me.  
In this case, we say that the edges are [undirected](https://en.wikipedia.org/wiki/Graph_%28discrete_mathematics%29#Undirected_graph).  
Hyperlinks are not -- a website $A$ may link to website $Z$ without $Z$ linking to $A$.  
We call these types of edges [directed](https://en.wikipedia.org/wiki/Graph_%28discrete_mathematics%29#Directed_graph).  
We'll be dealing with both kinds of networks.

## Betweenness Centrality

In Chapter 1, we calculated the key connectors in the DataSciencester network by counting the number of friends that each user had.  
Now we have enough machinery to look at other approaches.  
Recall that the network comprised users:

In [20]:
users = [
    { "id": 0, "name": "Hero" },
    { "id": 1, "name": "Dunn" },
    { "id": 2, "name": "Sue" },
    { "id": 3, "name": "Chi" },
    { "id": 4, "name": "Thor" },
    { "id": 5, "name": "Clive" },
    { "id": 6, "name": "Hicks" },
    { "id": 7, "name": "Devin" },
    { "id": 8, "name": "Kate" },
    { "id": 9, "name": "Klein" }
]

and friendships:

In [21]:
friendships = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
               (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

This is what our network looks like:

![users_network_graph](img/users_network_graph.png)

We also added friend lists to each user `dict`:

In [22]:
for user in users:
    user["friends"] = []
    
for i, j in friendships:
    # this works because users[i] is the user whose id is i 
    users[i]["friends"].append(users[j])  # add i as a friend of j
    users[j]["friends"].append(users[i])  # add j as a friend of i

When we left off we were dissatisfied with our notion of [degree centrality](https://en.wikipedia.org/wiki/Centrality#Degree_centrality), which didn't really agree with our intuition about who were the key connectors of the network.  
An alternative metric is [betweenness centrality](https://en.wikipedia.org/wiki/Centrality#Betweenness_centrality), which identifies people who frequently are on the shortest paths between pairs of other people.  
In particular, the betweenness centrality of node $i$ is computed by adding up, for every other pair of nodes $j$ and $k$, the proportion of shortest paths between node $j$ and node $k$ that pass through $i$.

As an example, to figure out Thor's betweenness centrality, we'll need to compute all of the shortest paths between all pairs of people who aren't Thor.  
Then we'll need to count how many of those shortest paths pass through Thor.  
For instance, the only shortest path between Chi (id 3) and Clive (id 5) passes through Thor, while neither of the shortest paths between Hero (id 0) and Chi (id 3) does.
Therefore, as a first step, we'll need to figure out the shortest paths between all pairs of people.  
There are some pretty sophisticated algorithms for doing so efficiently, but, because we're learning and building from scratch, we'll use a less efficient but more easily understood algorithm.

This algorithm, which is an implementation of [breadth-first search](https://en.wikipedia.org/wiki/Breadth-first_search), is one of the more complicated ones in this book, so let's go through it carefully:
1. Our goal is a function that takes a `from_user` and finds *all* shortest paths to every other user.
2. We'll represent a path as a `list` of user IDs. Since every path starts at `from_user`, we won't include her ID in the list. This means that the length of the list representing the path will be the length of the path itself.
3. We'll maintain a dictionary `shortest_paths_to` where the keys are user IDs and the values are lists of paths that end at the user with the specified ID. If there is a unique shortest path, the list will just contain that one path. If there are multiple shortest paths, the list will contain all of them.
4. We'll also maintain a [queue](https://en.wikipedia.org/wiki/Breadth-first_search) `frontier` that contains the users we want to explore in the order we want to explore them. (A queue is a data structure that is optimized for operations that "add to the end" and "remove from the front". In Python, they are implemented as [collections.deque](https://docs.python.org/2/library/collections.html#collections.deque) which is actually a double-ended queue.) We'll store them as pairs (`prev_user, user`) so that we know how we got to each one. We initialize the queue with all of the neighbors of `from_user`.
5. As we explore the graph, whenever we find new neighbors that we don't already know the shortest paths to, we add them to the end of the queue to explore later, with the current user as `prev_user`.
6. When we take a user off of the queue, and we've never encountered that user before, we've definitely found one or more shortest paths to him -- each of the shortest paths to `prev_user` with one extra step added.
7. When we take a user off of the queue and we *have* encountered that user before, then either we've found another shortest path (in which case we should add it) or we've found a longer path (in which case we shouldn't).
8. When no more users are left on the queue, we've explored the whole graph (or, at least. the parts of it that are reachable from the starting user) and we're done.

We can put all of this together into one, rather large, function:

In [23]:
def shortest_paths_from(from_user):
    
    # a dictionary from "user_id" to *all* shortest paths to that user
    shortest_paths_to = { from_user["id"] : [[]] }

    # a queue of (previous user, next user) that we need to check.
    # starts out with all pairs (from_user, friend_of_from_user)
    frontier = deque((from_user, friend)
                     for friend in from_user["friends"])

    # keep going until we empty the queue
    while frontier: 

        prev_user, user = frontier.popleft() # take from the beginning
        user_id = user["id"]

        # the fact that we're pulling from our queue means that
        # necessarily we already know a shortest path to prev_user
        paths_to_prev = shortest_paths_to[prev_user["id"]]
        paths_via_prev = [path + [user_id] for path in paths_to_prev]
        
        # it's possible we already know a shortest path to here as well
        old_paths_to_here = shortest_paths_to.get(user_id, [])
        
        # what's the shortest path to here that we've seen so far?
        if old_paths_to_here:
            min_path_length = len(old_paths_to_here[0])
        else:
            min_path_length = float('inf')
                
        # any new paths to here that aren't too long
        new_paths_to_here = [path_via_prev
                             for path_via_prev in paths_via_prev
                             if len(path_via_prev) <= min_path_length
                             and path_via_prev not in old_paths_to_here]
        
        shortest_paths_to[user_id] = old_paths_to_here + new_paths_to_here
        
        # add new neighbors to the frontier
        frontier.extend((user, friend)
                        for friend in user["friends"]
                        if friend["id"] not in shortest_paths_to)

    return shortest_paths_to

Now we can store these `dicts` with each node:

In [24]:
for user in users:
    user["shortest_paths"] = shortest_paths_from(user)

Now we're finally ready to calculate betweenness centrality.  
For every pair of nodes $i$ and $j$, we know the $n$ shortest paths from $i$ to $j$.  
Then, for each of those paths, we just add $1/n$ to the centrality of each node on that path:

In [25]:
for user in users:
    user["betweenness_centrality"] = 0.0
    
for source in users:
    source_id = source["id"]
    for target_id, paths in source["shortest_paths"].iteritems():
        if source_id < target_id:    # don't double count
            num_paths = len(paths)   # how many shortest_paths?
            contrib = 1 / num_paths  # contribution to centrality
            for path in paths:
                for id in path:
                    if id not in [source_id, target_id]:
                        users[id]["betweenness_centrality"] += contrib

for user in users:
    print user["id"], user["betweenness_centrality"]

0 0.0
1 3.5
2 3.5
3 18.0
4 20.0
5 20.5
6 6.0
7 6.0
8 8.5
9 0.0


The author included a nice visualization to show the relationships:

![betweenness_centrality](img/betweenness_centrality.png)

As shown above, users 0 and 9 have centrality 0.0 (meaning that neither one is on any shortest path between other users), whereas users 3, 4, and 5 all have high centralities (meaning that all three lie on many shortest paths).  
Generally the centrality numbers are relative and aren't that meaningful themselves; what we care about is how the numbers for each node compare to the numbers for other nodes.

## Closeness Centrality

Another measure that we can look at is [closeness centrality](https://en.wikipedia.org/wiki/Centrality#Closeness_centrality).  
First, for each user we calculate her [farness](http://www.analytictech.com/ucinet/help/oeky47.htm), which is the sum of the lengths of her shortest paths to each other user.  
Since we've already computed the shortest paths between each pair of nodes, we can add up their lengths.  
If there are multiple shortest paths, they all have the same length, so we can just look at the first one.

In [26]:
def farness(user):
    """ the sum of the lengths of the shortest paths to each other user """
    return sum(len(paths[0]) for paths in user["shortest_paths"].values())

Now we can go on and calculate closeness centrality as follows:

In [27]:
for user in users:
    user["closeness_centrality"] = 1 / farness(user)
    print user["id"], user["closeness_centrality"]

0 0.0294117647059
1 0.037037037037
2 0.037037037037
3 0.0454545454545
4 0.05
5 0.05
6 0.0416666666667
7 0.0416666666667
8 0.0357142857143
9 0.0277777777778


Now here's another pretty picture courtesy of the author to complement our results:

![closeness_centrality](img/closeness_centrality.png)

There is much less variation here -- even the very central nodes are still pretty far from the out on the periphery.  
Calculating shortest paths can be a bit of a challenge, which is one reason why betweenness and closeness centrality aren't often used on large networks.  
The less intuitive (but generally easier to compute) [eigenvector centrality](https://en.wikipedia.org/wiki/Eigenvector_centrality) is more frequently used.

## Eigenvector Centrality